In [1]:
# Cài đặt các thư viện cần thiết
!pip uninstall -y protobuf
!pip install -q protobuf==3.20.0
!pip install -q transformers datasets accelerate sentencepiece
!pip install -q scikit-learn pandas numpy tqdm openpyxl

# Install bitsandbytes - QUAN TRỌNG: dùng -U để lấy latest version
!pip uninstall -y bitsandbytes
!pip install -U bitsandbytes
!pip install -q peft

# Suppress warnings
import os
import warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['GRPC_VERBOSITY'] = 'ERROR'
warnings.filterwarnings('ignore')

print("✅ Dependencies installed!")

Found existing installation: protobuf 6.33.0
Uninstalling protobuf-6.33.0:
  Successfully uninstalled protobuf-6.33.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 4.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 3.20.0 which is incompatible.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.0 which is incompatible.
google-cloud-secret-manager 2.25.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<7.0.0,>=3.20.2, but you have protobuf 3.20.0 which is incompatible.
google-cloud-vision 3.11.0 requires protobuf!=4.21.0,!=4.

In [2]:
# 🎮 GPU Auto-Detection & Optimization
import torch
import os

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print(f"\n{'='*70}")
    print(f"🎮 DETECTED: {num_gpus} GPU(s)")
    print(f"{'='*70}")
    
    for i in range(num_gpus):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"GPU {i}: {gpu_name}")
    
    # Auto-configure based on number of GPUs
    if num_gpus == 1:
        # Single GPU (P100)
        print(f"\n{'='*70}")
        print("⚙️  SINGLE GPU MODE (P100 Optimized)")
        print(f"{'='*70}")
        
        os.environ['CUDA_VISIBLE_DEVICES'] = '0'
        
        BATCH_SIZE_QWEN = 2  # P100 (16GB)
        
        GPU_CONFIG = {
            'num_gpus': 1,
            'device': 'cuda:0',
            'batch_sizes': {
                'qwen': BATCH_SIZE_QWEN
            },
            'gradient_accumulation_steps': 2
        }
        
    elif num_gpus >= 2:
        # Multi-GPU (T4 x2)
        print(f"\n{'='*70}")
        print("⚡ MULTI-GPU MODE (T4 x2 Maximum Performance)")
        print(f"{'='*70}")
        
        BATCH_SIZE_QWEN = 4  # T4 x2 (32GB)
        
        GPU_CONFIG = {
            'num_gpus': num_gpus,
            'device': 'cuda',
            'batch_sizes': {
                'qwen': BATCH_SIZE_QWEN
            },
            'gradient_accumulation_steps': 1
        }
    
    print(f"\n📊 Configured Batch Size:")
    print(f"   Qwen:   {GPU_CONFIG['batch_sizes']['qwen']}")
    print(f"\n{'='*70}\n")
    
else:
    print("❌ No GPU detected! This notebook requires GPU.")
    GPU_CONFIG = None

PyTorch version: 2.6.0+cu124
CUDA available: True

🎮 DETECTED: 2 GPU(s)
GPU 0: Tesla T4
GPU 1: Tesla T4

⚡ MULTI-GPU MODE (T4 x2 Maximum Performance)

📊 Configured Batch Size:
   Qwen:   4




In [3]:
import sys
import os

# Copy files ra working directory
!cp -r /kaggle/input/vithsd-experiment/* /kaggle/working/

os.chdir("/kaggle/working")
sys.path.insert(0, "/kaggle/working")

print("📁 Files copied to working directory")

📁 Files copied to working directory


In [4]:
# Import modules
from config import FINAL_LABELS, DATA_DIR, OUTPUT_DIR
from data_preparation import load_dataset_A
from evaluation import Evaluator
from models import create_model

print("✅ All modules imported!")
print(f"📊 Labels: {len(FINAL_LABELS)}")

✅ All modules imported!
📊 Labels: 11


In [5]:
# Load Dataset A
train_texts, train_labels = load_dataset_A('train')
test_texts, test_labels = load_dataset_A('test')

print(f"📊 Dataset A:")
print(f"   Train: {len(train_texts)} samples")
print(f"   Test: {len(test_texts)} samples")

[ViHSDLoader] Loaded 7000 samples from train
[ViHSDLoader] Loaded 1800 samples from test
📊 Dataset A:
   Train: 7000 samples
   Test: 1800 samples


In [6]:
# Step X: Load Dataset Rationale (dataset_rationale.json)
import json
from pathlib import Path

def load_dataset_rationale(
    split: str = "train",
    base_dir: str = "/kaggle/input/dataset-ie",
    filename: str = "dataset_rationale.json",
    require_implied: bool = True,
):
    """
    Load dataset_rationale.json and return:
      - contents: List[str]
      - implieds: List[str]
      - rationales: List[list[str] | str]  (tuỳ file bạn lưu)
      - labels: List[list[str]]  (nếu có)
      - ids: List[int|str]
    
    split: "train" | "dev" | "test"
    require_implied: nếu True thì chỉ lấy mẫu có implied_statement (để dùng stage 2 alignment)
    """
    path = Path(base_dir) / filename
    if not path.exists():
        raise FileNotFoundError(f"Cannot find {path}. Check Add Input dataset name/path.")

    with path.open("r", encoding="utf-8") as f:
        data = json.load(f)

    # data can be list or dict containing list
    if isinstance(data, dict):
        for k in ["data", "items", "results"]:
            if k in data and isinstance(data[k], list):
                records = data[k]
                break
        else:
            raise ValueError("dataset_rationale.json is a dict but no list found under keys: data/items/results")
    elif isinstance(data, list):
        records = data
    else:
        raise ValueError("dataset_rationale.json must be a list or a dict containing a list")

    split = str(split).lower().strip()
    if split not in {"train", "dev", "test"}:
        raise ValueError("split must be one of: train/dev/test")

    contents, implieds, rationales, labels, ids_ = [], [], [], [], []
    kept, skipped = 0, 0

    for r in records:
        r_split = str(r.get("dataset", "")).lower().strip()
        if r_split != split:
            continue

        content = (r.get("content") or "").strip()
        implied = (r.get("implied_statement") or "").strip()

        if not content:
            skipped += 1
            continue
        if require_implied and not implied:
            skipped += 1
            continue

        contents.append(content)
        implieds.append(implied)
        rationales.append(r.get("rationale", None))   # có thể là list[str] hoặc str
        labels.append(r.get("labels", []))            # list[str] nếu có
        ids_.append(r.get("id", None))

        kept += 1

    print(f"[RationaleLoader] Loaded {kept} samples for split='{split}' from: {path}")
    if skipped > 0:
        print(f"[RationaleLoader] Skipped {skipped} samples (missing content or implied_statement)")

    return contents, implieds, rationales, labels, ids_


# --- Example usage (Stage 2 should use train only to avoid leakage) ---
r_train_texts, r_train_implied, r_train_rat, r_train_labels, r_train_ids = load_dataset_rationale("train")

print("Example:")
print("content:", r_train_texts[0][:120])
print("implied:", r_train_implied[0][:120])
print("rationale:", r_train_rat[0])
print("labels:", r_train_labels[0])

[RationaleLoader] Loaded 1221 samples for split='train' from: /kaggle/input/dataset-ie/dataset_rationale.json
Example:
content: ăn bom trên nhà thằng adm trốc tôm thôi
implied: admin ngu dốt
rationale: ["Cụm từ 'thằng adm' trực tiếp chỉ định đối tượng là admin với lời nói mang tính miệt thị.", "Cách dùng 'trốc tôm' nhằm hạ thấp phẩm giá và trí tuệ của admin.", 'Cách diễn đạt ngắn gọn, cắt nghĩa làm tăng tính tấn công của lời nói.', 'Ngôn từ tổng thể phản ánh định kiến cho rằng admin là người ngu dốt.']
labels: ['individuals#hate']


In [7]:
sum(1 for x in r_train_implied if not x or not x.strip())

0

In [8]:
# =========================
# Cell 6: OPTIMIZED - Fast Training Pipeline (NO DEV)
# =========================

# Step 1: Initialize Evaluator
evaluator = Evaluator()
print("✅ Evaluator initialized")

# Step 2: Skip dev set - set flag to False
has_dev = False  # 🔧 FIX: Explicitly set to False
print("⚠️ Skipping dev set evaluation")

# Step 3: Build alignment pairs from rationale (TRAIN ONLY)
print("\n" + "="*70)
print("📚 Building Stage 2 Alignment Pairs (Train-only)")
print("="*70)

r_pairs = []
for content, implied in zip(r_train_texts, r_train_implied):
    if content.strip() and implied.strip():
        r_pairs.append({
            'content': content,
            'implied': implied
        })

print(f"✅ Built {len(r_pairs)} alignment pairs from train rationale")

# Step 4: Create model with OPTIMIZED parameters
print("\n" + "="*70)
print("🚀 Creating Qwen Model (OPTIMIZED)")
print("="*70)

# ⚡ OPTIMIZATION: 2 epochs as you specified
model_qwen = create_model(
    "qwen",
    dataset_type="A",
    batch_size=GPU_CONFIG['batch_sizes']['qwen'],  # batch_size=4 for T4x2
    num_epochs=2,          # 2 epochs as you want
    learning_rate=2e-4,    # Stage 1 lr
    lora_r=8,
    lora_alpha=16
)


✅ Evaluator initialized
⚠️ Skipping dev set evaluation

📚 Building Stage 2 Alignment Pairs (Train-only)
✅ Built 1221 alignment pairs from train rationale

🚀 Creating Qwen Model (OPTIMIZED)


In [9]:

# =========================
# STAGE 1: Train classifier on original dataset
# =========================
print("\n" + "="*70)
print("🎯 STAGE 1: Training Classifier on Dataset A (2 epochs)")
print("="*70)

model_qwen.train(train_texts, train_labels)

# Skip dev evaluation (has_dev = False)
print("⚠️ Skipping dev evaluation (no dev set)")

# =========================
# STAGE 2: Semantic alignment using rationale
# =========================
print("\n" + "="*70)
print("🔗 STAGE 2: Semantic Alignment with Rationale")
print("="*70)

# Check if model has Stage 2 method
if hasattr(model_qwen, "train_stage2_alignment") and callable(getattr(model_qwen, "train_stage2_alignment")):
    try:
        model_qwen.train_stage2_alignment(
            r_pairs,
            num_epochs=1,
            learning_rate=5e-5  # Lower LR to avoid catastrophic forgetting
        )
        print("✅ Stage 2 alignment completed")
        
        # Skip dev eval after Stage 2 (has_dev = False)
        print("⚠️ Skipping dev evaluation (no dev set)")
            
    except Exception as e:
        print(f"⚠️ Stage 2 alignment failed: {e}")
        print("   Continuing with Stage 1 model only...")
else:
    print("⚠️ Model does not have train_stage2_alignment() method")
    print("   Skipping Stage 2 - using Stage 1 model only")

# =========================
# TEST EVALUATION (Final)
# =========================
print("\n" + "="*70)
print("🎯 FINAL TEST EVALUATION")
print("="*70)

pred_test, _ = model_qwen.predict(test_texts)
res_test = evaluator.evaluate(test_labels, pred_test, "Qwen", "A_test_final")
evaluator.print_result(res_test)

print("\n✅ Training pipeline completed successfully!")
print(f"📊 Final Test Results:")
print(f"   Subset Accuracy: {res_test['subset_accuracy']:.4f}")
print(f"   Macro F1: {res_test['f1_macro']:.4f}")
print(f"   Micro F1: {res_test['f1_micro']:.4f}")

# 🕐 Estimated total time: ~4 hours (Stage 1: 2 epochs) + ~30 min (Stage 2) = ~4.5 hours
# Well within Kaggle's 9-hour limit


🎯 STAGE 1: Training Classifier on Dataset A (2 epochs)


E0000 00:00:1765964595.981355      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765964596.087484      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[Qwen2.5-3B_MultiLabel] Training with QLoRA (4-bit)...
  Model: Qwen/Qwen2.5-3B-Instruct
  Device: cuda
  Original samples: 7000
  Mode: Standard (Dataset A)
  [Oversampling] Added 540 samples for minority labels
  LoRA config: r=8, alpha=16


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

  Epoch 1: avg_loss = 0.2475


Epoch 2/2: 100%|██████████| 1885/1885 [2:14:14<00:00,  4.27s/it, loss=0.1980]


  Epoch 2: avg_loss = 0.1905
  ✓ Training completed
⚠️ Skipping dev evaluation (no dev set)

🔗 STAGE 2: Semantic Alignment with Rationale
⚠️ Model does not have train_stage2_alignment() method
   Skipping Stage 2 - using Stage 1 model only

🎯 FINAL TEST EVALUATION


Predicting: 100%|██████████| 1800/1800 [26:52<00:00,  1.12it/s]


📊 Qwen on Dataset A_test_final (Multi-Label)
  Subset Accuracy (Exact Match): 0.5644
  Hamming Loss:                  0.0852
  
  F1 Samples:                    0.6345
  F1 Macro:                      0.3035
  F1 Micro:                      0.6026
  
  Precision (Samples):           0.6515
  Precision (Macro):             0.4067
  Precision (Micro):             0.6347
  
  Recall (Samples):              0.6344
  Recall (Macro):                0.2842
  Recall (Micro):                0.5735

  F1 per Label:
    normal                   : 0.7984
    individuals#offensive    : 0.2014
    individuals#hate         : 0.6166
    groups#offensive         : 0.0788
    groups#hate              : 0.5344
    race#offensive           : 0.2264
    race#hate                : 0.2817
    politics#offensive       : 0.0606
    politics#hate            : 0.5400

✅ Training pipeline completed successfully!
📊 Final Test Results:
   Subset Accuracy: 0.5644
   Macro F1: 0.3035
   Micro F1: 0.6026


In [10]:
# Step 5: Save Results (after Stage 2)
import json
from pathlib import Path
import numpy as np

output_dir = Path("/kaggle/working/outputs")
output_dir.mkdir(exist_ok=True)

def convert_to_serializable(obj):
    if isinstance(obj, dict):
        return {k: convert_to_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, (list, tuple)):
        return [convert_to_serializable(item) for item in obj]
    elif isinstance(obj, (np.integer, np.floating)):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    return obj

# ✅ dùng kết quả test cuối
final_result = res_test  # <--- quan trọng

results = {'Qwen': convert_to_serializable(final_result)}
with open(output_dir / "results_datasetA_qwen_stage2.json", "w") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print("✅ Results saved to /kaggle/working/outputs/results_datasetA_qwen_stage2.json")
print(f"\n📊 Qwen Results:")
print(f"   Subset Accuracy: {final_result['subset_accuracy']:.4f}")
print(f"   Macro F1: {final_result['f1_macro']:.4f}")
print(f"   Micro F1: {final_result['f1_micro']:.4f}")


✅ Results saved to /kaggle/working/outputs/results_datasetA_qwen_stage2.json

📊 Qwen Results:
   Subset Accuracy: 0.5644
   Macro F1: 0.3035
   Micro F1: 0.6026


In [11]:
# Step 6: Save Model State Dict (after Stage 2)
import torch
from pathlib import Path

checkpoint_dir = Path("/kaggle/working/checkpoints")
checkpoint_dir.mkdir(exist_ok=True)

# ✅ dùng kết quả test cuối
final_result = res_test  # <--- quan trọng

lora_adapter_dir = checkpoint_dir / "qwen_datasetA_stage2_lora_adapters"
model_qwen.model.save_pretrained(lora_adapter_dir)

tokenizer_dir = checkpoint_dir / "qwen_datasetA_stage2_tokenizer"
model_qwen.tokenizer.save_pretrained(tokenizer_dir)

metadata_path = checkpoint_dir / "qwen_datasetA_stage2_metadata.pt"
torch.save({
    'model_config': {
        'base_model': 'Qwen/Qwen2.5-3B-Instruct',
        'quantization': '4-bit',
        'lora_config': {
            'r': 8,
            'lora_alpha': 16,
            'target_modules': ['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj']
        },
        'labels': getattr(model_qwen, "labels", None)
    },
    'training_info': {
        'dataset': 'A',
        'technique': 'QLoRA (4-bit + LoRA) + Stage2 Semantic Alignment',
        'subset_accuracy': final_result['subset_accuracy'],
        'macro_f1': final_result['f1_macro'],
        'micro_f1': final_result['f1_micro']
    }
}, metadata_path)

print(f"✅ LoRA adapters saved to: {lora_adapter_dir}")
print(f"✅ Tokenizer saved to: {tokenizer_dir}")
print(f"✅ Metadata saved to: {metadata_path}")


✅ LoRA adapters saved to: /kaggle/working/checkpoints/qwen_datasetA_stage2_lora_adapters
✅ Tokenizer saved to: /kaggle/working/checkpoints/qwen_datasetA_stage2_tokenizer
✅ Metadata saved to: /kaggle/working/checkpoints/qwen_datasetA_stage2_metadata.pt
